In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from models_comp import SynthesizerTrnComp
from text.symbols import symbols
from text import text_to_generalized_sequence


def get_text(text, hps):
    text_norm = text_to_generalized_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    print(text_norm)
    return text_norm

In [7]:
hps = utils.get_hparams_from_file("./logs/bird01_cath/config.json")

In [8]:
net_g = SynthesizerTrnComp(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model).cuda()
_ = net_g.eval()

_ = utils.load_checkpoint("logs/bird01_cath/G_9000.pth", net_g, None)

In [9]:
txt='SIL^ala^{"ai":0.5,"alc":0.5}^ala^{"ai":3,"hi":-1}^SIL'
x_tst = get_text(txt, hps)
with torch.no_grad():
    x_tst_lengths = torch.LongTensor([len(x_tst)]).cuda()
    audio = net_g.inferComp(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

[0, 140, 0, 1, 0, {13: 0.5, 2: 0.5}, 0, 1, 0, {13: 3, 107: -1}, 0, 140, 0]
